<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/12-Image%20and%20Video%20Analytics/K_Means_to_find_the_dominant_Colors_in_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://charlesleifer.com/blog/using-python-and-k-means-to-find-the-dominant-colors-in-images/

In [2]:
!wget https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Images/akira.jpg
!wget https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Images/akira-cycle-2.png
!wget https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Images/akira-neo-tokyo-6.png
!wget https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Images/akira-neo-tokyo-7.png

--2021-08-04 20:19:17--  https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Images/akira.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209243 (204K) [image/jpeg]
Saving to: ‘akira.jpg’

akira.jpg           100%[===================>] 204.34K  --.-KB/s    in 0.03s   

2021-08-04 20:19:17 (7.36 MB/s) - ‘akira.jpg’ saved [209243/209243]

--2021-08-04 20:19:17--  https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Images/akira-cycle-2.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 564320 (5

In [1]:
from collections import namedtuple
from math import sqrt
import random
try:
    import Image
except ImportError:
    from PIL import Image

Point = namedtuple('Point', ('coords', 'n', 'ct'))
Cluster = namedtuple('Cluster', ('points', 'center', 'n'))

def get_points(img):
    points = []
    w, h = img.size
    for count, color in img.getcolors(w * h):
        points.append(Point(color, 3, count))
    return points

rtoh = lambda rgb: '#%s' % ''.join(('%02x' % p for p in rgb))

def colorz(filename, n=3):
    img = Image.open(filename)
    img.thumbnail((200, 200))
    w, h = img.size

    points = get_points(img)
    clusters = kmeans(points, n, 1)
    rgbs = [map(int, c.center.coords) for c in clusters]
    return map(rtoh, rgbs)

def euclidean(p1, p2):
    return sqrt(sum([
        (p1.coords[i] - p2.coords[i]) ** 2 for i in range(p1.n)
    ]))

def calculate_center(points, n):
    vals = [0.0 for i in range(n)]
    plen = 0
    for p in points:
        plen += p.ct
        for i in range(n):
            vals[i] += (p.coords[i] * p.ct)
    return Point([(v / plen) for v in vals], n, 1)

def kmeans(points, k, min_diff):
    clusters = [Cluster([p], p, p.n) for p in random.sample(points, k)]

    while 1:
        plists = [[] for i in range(k)]

        for p in points:
            smallest_distance = float('Inf')
            for i in range(k):
                distance = euclidean(p, clusters[i].center)
                if distance < smallest_distance:
                    smallest_distance = distance
                    idx = i
            plists[idx].append(p)

        diff = 0
        for i in range(k):
            old = clusters[i]
            center = calculate_center(plists[i], old.n)
            new = Cluster(plists[i], center, old.n)
            clusters[i] = new
            diff = max(diff, euclidean(old.center, new.center))

        if diff < min_diff:
            break

    return clusters